In [1]:
# Refactorise apl pour toutes les professions
# Créer le schéma apl et mettre toutes les stats dedans
# Faire la table de correspondance specialite PSP_SPE_SNDS
# tarif_stats ne fonctionne pas
# Faire sur damir_acte_stats : ROW GROUP BY PSP_SPE_SNDS
# Faire damir_age_stage : ROW GROUP BY AGE_BEN_SNDS, PSP_SPE_SNDS

In [13]:
import pandas as pd
import numpy as np
import config
from sqlalchemy import text
print(config.version)
print(config.connection_string)
pd.set_option('display.max_columns', None)

3.2.0.Dev
postgresql://postgres:sa@localhost/icip


In [3]:
specialite=10
year=21
dept=6
source="PS"
min=30
print(year, specialite, source)

21 10 PS


In [4]:
sources: dict[str, str] = {
    "PS": f"""
select ps.id, ps.nom, ps.prenom, an.dept_id, an.id adresse_norm_id, i.id iris, an.lon, an.lat from ps
join tarif t on t.ps_id = ps.id
join tarif_date_source tds on tds.tarif_id=t.id 
join cabinet c on t.cabinet_id=c.id
join adresse_raw ar on c.adresse_raw_id=ar.id
join adresse_norm an on ar.adresse_norm_id=an.id
join specialite_profession sp on sp.profession_id=t.profession_id
join iris.iris i on i.code=an.iris
where sp.specialite_id={specialite}
and tds.date_source_id >= {year}00 and  tds.date_source_id < {year+1}00
and ar.dept_id={dept}
group by ps.id, c.id, an.id, i.id
""",
    "PA": f"""
select pa.id, pa.nom, pa.prenom, an.dept_id, an.id adresse_norm_id, i.id iris, an.lon, an.lat from personne_activite pa
join personne_activite_diplome pad on pad.personne_activite_id=pa.id
join diplome d on pad.diplome_id=d.id
join profession_diplome pd on pd.diplome_id=d.id
join profession p on pd.profession_id=p.id
join specialite_profession sp on sp.profession_id=p.id
join pa_adresse_norm_date_source pands ON pands.personne_activite_id = pa.id
join adresse_norm an on pands.adresse_norm_id=an.id
join iris.iris i on i.code=an.iris
where sp.specialite_id={specialite}
and pands.date_source_id/100={year}
and an.dept_id={dept}
group by pa.id, an.id, i.id
""",
    "RPPS": f"""
select personne.id, personne.nom, personne.prenom, an.dept_id, an.id adresse_norm_id, i.id iris, an.lon, an.lat from personne
join diplome_obtenu dio on dio.personne_id = personne.id
join diplome d on d.id=dio.diplome_id
join profession_diplome pd on pd.diplome_id=d.id
join profession p on pd.profession_id=p.id
join specialite_profession sp on sp.profession_id=p.id
join coord c on c.personne_id=personne.id
join adresse_norm an on c.adresse_norm_id=an.id
join iris.iris i on i.code=an.iris
where sp.specialite_id={specialite}
and personne.date_maj > '20{year}-01-01' and date_effet < '20{year}-01-01'
and an.dept_id={dept}
group by personne.id, an.id, i.id
"""}

sql = sources[source]
print(sql)


select ps.id, ps.nom, ps.prenom, an.dept_id, an.id adresse_norm_id, i.id iris, an.lon, an.lat from ps
join tarif t on t.ps_id = ps.id
join tarif_date_source tds on tds.tarif_id=t.id 
join cabinet c on t.cabinet_id=c.id
join adresse_raw ar on c.adresse_raw_id=ar.id
join adresse_norm an on ar.adresse_norm_id=an.id
join specialite_profession sp on sp.profession_id=t.profession_id
join iris.iris i on i.code=an.iris
where sp.specialite_id=10
and tds.date_source_id >= 2100 and  tds.date_source_id < 2200
and ar.dept_id=6
group by ps.id, c.id, an.id, i.id



In [5]:
df = pd.read_sql(sql, config.connection_string)
df

,id,nom,prenom,dept_id,adresse_norm_id,iris,lon,lat
0,16126,LEONARDI,DOMINIQUE,6,1415,60881903,7.255881,43.709653
1,16130,BENSUSSAN,JEAN,6,92990,60882801,7.227639,43.676026
2,16131,LARA,JEANNINE,6,92963,60880202,7.282865,43.695573
3,16132,GUEGAN,JEAN CLAUDE,6,1425,60270102,7.145627,43.657352
4,16148,CROQUET,PATRICE,6,1433,60690109,6.922279,43.651604
...,...,...,...,...,...,...,...,...
1280,155689,SAROUX,DELPHINE,6,2305,60882401,7.258125,43.695605
1281,155690,PERADOTTO,CYRIL,6,2866,60290121,6.988390,43.550361
1282,155692,DONADIO,MANON,6,77494,61490102,7.313438,43.743446
1283,155827,BOUCHET BADEL,MARIE,6,2328,60690102,6.914032,43.646318


In [6]:
nb_ps = df["id"].nunique()
unique_ps = df.groupby(["id", "lon", "lat"])
print(f"Nb PS {nb_ps}")
print(f"Nb unique PS {len(unique_ps)}")

Nb PS 1203
Nb unique PS 1273


In [7]:
df["key"] = df["id"].astype(str) + "_" + df["lat"].astype(str) + "_" + df["lon"].astype(str)
df["nb_cabinet"] = df.groupby("id")["key"].transform("nunique")
df["nb_cabinet"].value_counts(normalize=True)

nb_cabinet
1    0.893385
2    0.094163
3    0.009339
4    0.003113
Name: proportion, dtype: float64

In [8]:
df["weight"] = 1 / df["nb_cabinet"]
df["nb_per_iris"] = df.groupby("iris")["weight"].transform("sum")
df.head(10)

,id,nom,prenom,dept_id,adresse_norm_id,iris,lon,lat,key,nb_cabinet,weight,nb_per_iris
0,16126,LEONARDI,DOMINIQUE,6,1415,60881903,7.255881,43.709653,16126_43.709653_7.255881,1,1.0,5.000000
1,16130,BENSUSSAN,JEAN,6,92990,60882801,7.227639,43.676026,16130_43.676026_7.227639,1,1.0,8.000000
2,16131,LARA,JEANNINE,6,92963,60880202,7.282865,43.695573,16131_43.695573_7.282865,1,1.0,3.000000
3,16132,GUEGAN,JEAN CLAUDE,6,1425,60270102,7.145627,43.657352,16132_43.657352_7.145627,1,1.0,5.000000
4,16148,CROQUET,PATRICE,6,1433,60690109,6.922279,43.651604,16148_43.651604_6.922279,1,1.0,10.500000
5,16151,DESGATS,MARC,6,1436,60300107,7.005844,43.579045,16151_43.579045_7.005844,1,1.0,5.000000
6,16153,DUBOIS,MICHEL,6,1438,61360000,7.449651,43.877046,16153_43.877046_7.449651,1,1.0,4.000000
7,16159,GIDEL,PIERRE JEAN,6,1466,60882601,7.242220,43.689322,16159_43.689322_7.24222,1,1.0,11.666667
8,16160,MASCHINO,JEAN MARC,6,1467,60290102,7.037066,43.546353,16160_43.546353_7.037066,1,1.0,3.000000
9,16161,SARDOU,YVES,6,1468,60830205,7.492397,43.791639,16161_43.791639_7.492397,1,1.0,1.500000


In [9]:
df = df.sort_values(by='iris')
df = df.drop_duplicates(subset=['iris', 'nb_per_iris'])
df


,id,nom,prenom,dept_id,adresse_norm_id,iris,lon,lat,key,nb_cabinet,weight,nb_per_iris
123,16425,DAEMERS,YVES,6,92989,60040102,7.072160,43.596934,16425_43.596934_7.07216,1,1.0,2.000000
904,113924,DIAMANT,GUY,6,77467,60040103,7.137517,43.567521,113924_43.567521_7.137517,1,1.0,1.000000
109,16404,MANDRILE,DIDIER,6,1651,60040104,7.092388,43.604533,16404_43.604533_7.092388,1,1.0,1.000000
31,16222,NICASTRO,SERGE,6,92975,60040105,7.117538,43.588105,16222_43.588105_7.117538,1,1.0,2.000000
1179,144024,FRICKEY,SARAH,6,2734,60040106,7.112369,43.606074,144024_43.606074_7.112369,1,1.0,8.666667
...,...,...,...,...,...,...,...,...,...,...,...,...
607,17388,ROUSSEAU,FRANCOISE,6,2635,61610103,7.123625,43.658047,17388_43.658047_7.123625,1,1.0,1.000000
1069,132333,AGOSTINI,PIERRE,6,81986,61610104,7.141591,43.641591,132333_43.641591_7.141591,2,0.5,3.000000
742,99028,BENASSAYAG,JEAN JACQUES,6,68080,61610105,7.140329,43.639824,99028_43.639824_7.140329,1,1.0,5.000000
301,16814,DI VINCENZO,DOMINIQUE,6,273951,61620000,7.612673,44.063665,16814_44.063665_7.612673,2,0.5,0.500000


In [34]:
def get_iris_matrix():
    sql = f"""
(select iris.id "iris1", iris.id "iris2", 0 "km", 0 "time" from iris.iris
where id / 10000000 = {dept}

union

select iris_id_from as "iris1", iris_id_to "iris2", route_km "km", route_min "time" from iris.iris_matrix
where (iris_id_from/10000000={dept} or iris_id_to/10000000={dept})
and route_min <= {min}) order by "IRIS1", "IRIS2"
"""
    print(sql)
    return pd.read_sql(sql, config.connection_string)

# iris_matrix = get_iris_matrix()
iris_matrix = pd.read_csv("od_06_sql_rename.csv")
iris_matrix["iris"] = iris_matrix["iris2"].astype("int64")
iris_matrix
    

,iris1,iris2,km,time,iris
0,40080000,60530000,17,22,60530000
1,40080000,60990000,21,24,60990000
2,40390000,61540000,24,30,61540000
3,40420000,60530000,11,19,60530000
4,40420000,60710000,22,30,60710000
...,...,...,...,...,...
117764,831380000,60850103,34,29,60850103
117765,831380000,60950101,18,19,60950101
117766,831380000,60950102,19,22,60950102
117767,831380000,60950103,18,20,60950103


In [35]:
def get_pop_iris():
    sql = f"""
select i.id iris, pi.iris iris_string, c.code code_commune, i.type type_iris, pi.p20_pop0002, pi.p20_pop0305, pi.p20_pop0610, pi.p20_pop1117, pi.p20_pop1824, pi.p20_pop2539, pi.p20_pop4559, pi.p20_pop6074, pi.p20_pop75p
from iris.pop_iris pi
join iris.iris i on  pi.iris=i.code
join iris.commune c on i.commune_id=c.id
where com like '{dept:02}%'"""
    print(sql)
    return pd.read_sql(text(sql), config.connection_string)

pop_iris = get_pop_iris()
pop_iris


select i.id iris, pi.iris iris_string, c.code code_commune, i.type type_iris, pi.p20_pop0002, pi.p20_pop0305, pi.p20_pop0610, pi.p20_pop1117, pi.p20_pop1824, pi.p20_pop2539, pi.p20_pop4559, pi.p20_pop6074, pi.p20_pop75p
from iris.pop_iris pi
join iris.iris i on  pi.iris=i.code
join iris.commune c on i.commune_id=c.id
where com like '06%'


,iris,iris_string,code_commune,type_iris,p20_pop0002,p20_pop0305,p20_pop0610,p20_pop1117,p20_pop1824,p20_pop2539,p20_pop4559,p20_pop6074,p20_pop75p
0,60120105,060120105,06012,A,87.418550,60.438694,193.693660,273.95730,223.035700,676.78790,811.84125,457.24982,315.660160
1,60500000,060500000,06050,C,14.457494,14.409695,41.336414,48.39683,17.661858,72.61527,136.39720,95.22634,43.630077
2,60720000,060720000,06072,C,0.000000,7.000000,5.000000,5.00000,4.000000,22.00000,38.00000,35.00000,26.000000
3,61050101,061050101,06105,A,90.654580,136.365310,244.816740,402.70312,211.586750,430.94614,945.29596,679.35130,356.906370
4,60150000,060150000,06015,C,36.382305,39.335630,53.229595,83.62182,64.078064,198.46510,316.26556,265.24164,108.909730
...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,60883403,060883403,06088,A,71.638420,55.635365,104.755210,138.66922,184.708200,294.65110,278.43094,337.78094,311.285340
496,60883501,060883501,06088,A,63.991177,86.144790,123.785240,215.96701,182.714100,349.32108,617.75810,466.61243,237.725800
497,60883601,060883601,06088,A,100.090430,73.343470,134.026950,215.99605,173.736830,455.11038,587.17010,585.84150,294.802900
498,60883602,060883602,06088,A,64.699010,79.082280,140.378510,183.99406,110.133000,398.01132,453.39966,453.67694,179.870680


In [36]:
apl = iris_matrix.merge(pop_iris, on="iris", how="left", suffixes=('', ''))
apl

,iris1,iris2,km,time,iris,iris_string,code_commune,type_iris,p20_pop0002,p20_pop0305,p20_pop0610,p20_pop1117,p20_pop1824,p20_pop2539,p20_pop4559,p20_pop6074,p20_pop75p
0,40080000,60530000,17,22,60530000,060530000,06053,C,3.000000,3.000000,3.000000,15.00000,3.000000,19.00000,23.00000,40.00000,26.000000
1,40080000,60990000,21,24,60990000,060990000,06099,C,45.961033,60.562077,118.186980,174.26068,100.920586,288.92783,365.60370,358.14850,209.747850
2,40390000,61540000,24,30,61540000,061540000,06154,C,14.443875,11.072821,23.354239,41.09353,36.612030,60.12047,105.71244,118.53387,45.796837
3,40420000,60530000,11,19,60530000,060530000,06053,C,3.000000,3.000000,3.000000,15.00000,3.000000,19.00000,23.00000,40.00000,26.000000
4,40420000,60710000,22,30,60710000,060710000,06071,C,13.000000,11.000000,30.000000,25.00000,21.000000,92.00000,118.00000,153.00000,90.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117764,831380000,60850103,34,29,60850103,060850103,06085,A,71.068900,71.788710,146.845930,194.39722,148.091400,351.81476,559.87990,510.06640,392.299470
117765,831380000,60950101,18,19,60950101,060950101,06095,A,78.000000,88.000000,211.000000,256.00000,155.000000,476.00000,682.00000,642.00000,411.000000
117766,831380000,60950102,19,22,60950102,060950102,06095,A,68.000000,78.000000,156.000000,226.00000,145.000000,400.00000,530.00000,485.00000,293.000000
117767,831380000,60950103,18,20,60950103,060950103,06095,A,47.000000,69.000000,126.000000,225.00000,103.000000,259.00000,567.00000,576.00000,327.000000


In [44]:
accessibilite_fn = lambda x: np.exp(-0.12 * x)

apl["accessibilite_weigth"] = accessibilite_fn(apl["time"])
apl.head(10)

,iris1,iris2,km,time,iris,iris_string,code_commune,type_iris,p20_pop0002,p20_pop0305,p20_pop0610,p20_pop1117,p20_pop1824,p20_pop2539,p20_pop4559,p20_pop6074,p20_pop75p,accessibilite_weigth,accessibilite_weigth2
0,40080000,60530000,17,22,60530000,060530000,06053,C,3.000000,3.000000,3.000000,15.000000,3.000000,19.000000,23.000000,40.000000,26.000000,0.071361,0.071361
1,40080000,60990000,21,24,60990000,060990000,06099,C,45.961033,60.562077,118.186980,174.260680,100.920586,288.927830,365.603700,358.148500,209.747850,0.056135,0.056135
2,40390000,61540000,24,30,61540000,061540000,06154,C,14.443875,11.072821,23.354239,41.093530,36.612030,60.120470,105.712440,118.533870,45.796837,0.027324,0.027324
3,40420000,60530000,11,19,60530000,060530000,06053,C,3.000000,3.000000,3.000000,15.000000,3.000000,19.000000,23.000000,40.000000,26.000000,0.102284,0.102284
4,40420000,60710000,22,30,60710000,060710000,06071,C,13.000000,11.000000,30.000000,25.000000,21.000000,92.000000,118.000000,153.000000,90.000000,0.027324,0.027324
5,40420000,60990000,17,24,60990000,060990000,06099,C,45.961033,60.562077,118.186980,174.260680,100.920586,288.927830,365.603700,358.148500,209.747850,0.056135,0.056135
6,40420000,61240000,15,30,61240000,061240000,06124,C,0.941176,0.000000,1.882353,0.941176,0.941176,2.823529,11.294118,16.941177,10.352942,0.027324,0.027324
7,40430000,60020000,15,20,60020000,060020000,06002,C,0.962963,0.000000,0.962963,0.000000,4.814815,2.888889,15.407408,15.407408,9.629630,0.090718,0.090718
8,40430000,60240000,8,11,60240000,060240000,06024,C,4.785714,5.982143,4.785714,17.946428,3.589286,21.535715,34.696430,52.642857,41.875000,0.267135,0.267135
9,40430000,60450000,18,23,60450000,060450000,06045,C,0.000000,0.000000,2.730769,6.371795,1.820513,2.730769,20.025640,21.846153,10.923077,0.063292,0.063292
